In [1]:
import multiprocessing
import re
from io import StringIO

import pandas as pd
import numpy as np

from emukit.core import ParameterSpace, ContinuousParameter
from emukit.core.initial_designs import RandomDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

In [2]:
target_name = 'SEIZ'
# n_runs = 5
# max_parallel_runs = 8
# threads_per_run = multiprocessing.cpu_count()//min(n_runs,max_parallel_runs)
threads_per_run = multiprocessing.cpu_count()

parameter_space = ParameterSpace([ContinuousParameter('p', 0, 1),
                                  ContinuousParameter('l', 0, 1),
                                  ContinuousParameter('epsilon', 2, 2)])

In [3]:
def run_job(p, l, epsilon, clear_results=False):
    with open(f'{target_name}_editable.fred', 'r') as file:
        filedata = file.read()

    for target, value in {'{p}': p, '{l}': l, '{epsilon}': epsilon}.items():
        filedata = filedata.replace(target, str(value))

    with open(f'{target_name}_out.fred', 'w') as file:
        file.write(filedata)
        
    if clear_results:
        !yes | fred_clear_all_results
        
    # !fred_job -p {target_name}_out.fred -k {target_name}_{p}_{l}_{epsilon}_run -n {n_runs} -m {max_parallel_runs} -t {threads_per_run}
    !fred_job -p {target_name}_out.fred -k {target_name}_{p}_{l}_{epsilon}_run -t {threads_per_run}
    !fred_csv -k {target_name}_{p}_{l}_{epsilon}_run > results/{target_name}_{p}_{l}_{epsilon}_run.csv

    # with open(f'results/{target_name}_{p}_{l}_{epsilon}_run.csv', 'r') as csv:
    #     results = csv.read()
    # 
    # results = re.split(' \nRun,\d\n', results)[1:]
    # return [pd.read_csv(StringIO(data)) for data in results]
    return pd.read_csv(f'results/{target_name}_{p}_{l}_{epsilon}_run.csv', header=1)

In [4]:
def sample(X, **kwargs):
    X = np.array(X)
    
    if X.ndim < 2:
        X = np.expand_dims(X, 0)
    elif X.ndim > 2:
        raise Exception(f'X has too many dimensions (ndim={X.ndim}, must be 1 or 2)')
    if X.shape[-1] != 3:
        raise Exception(f'X has the wrong number of variables (variables={X.shape[-1]}, must be 3)')
    
    Y = []

    for (p, l, epsilon) in X: # TODO: parallelize this?
        data = run_job(p, l, epsilon, **kwargs)
        # Y.append([run['ADOPT.I'].iat[-1] for run in data])
        Y.append([data['ADOPT.I'].iat[-1]])
        
    return np.vstack(Y)

In [5]:
design = RandomDesign(parameter_space)
num_data_points = 5
X = design.get_samples(num_data_points)

In [6]:
X

array([[0.14427996, 0.90493815, 2.        ],
       [0.97120135, 0.18209631, 2.        ],
       [0.36464294, 0.44585025, 2.        ],
       [0.78886984, 0.95171786, 2.        ],
       [0.44597143, 0.01819409, 2.        ]])

In [7]:
Y = sample(X, clear_results=True)

You are about to delete /home/nyoma/FRED/RESULTS. This cannot be undone.
Proceed? yes/no [no]
/home/nyoma/FRED/RESULTS deleted
yes: standard output: Broken pipe
fred_compile SEIZ_out.fred ...
No errors found.

FRED Warning (file SEIZ_out.fred) Ignoring duplicate rule:
  if state(SKEPTIC,E) then wait(24*normal(epsilon,epsilon/2))
     is hidden by:
  if state(SKEPTIC,E) then wait()

FRED Warning (file SEIZ_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_0.1442799638563732_0.9049381543842384_2.0_run 1
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/1/WORK/SEIZ_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/1/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/1/OUT/RUN1/LOG)
You are about to delete /home/nyoma/FRED/RESULTS. This cannot be undone.
Proceed? yes/no [no]
/home/nyoma/FRED/RESULTS deleted
yes: standard output: Broken pipe
fred_compile SEIZ_out.fred ...
No errors found.

FRED W

In [8]:
Y

array([[13820],
       [45219],
       [44146],
       [10469],
       [45250]])

In [9]:
model_gpy = GPRegression(X,Y)
model_emukit = GPyModelWrapper(model_gpy)

In [10]:
expected_improvement = ExpectedImprovement(model=model_emukit)

In [11]:
bayesopt_loop = BayesianOptimizationLoop(model=model_emukit,
                                         space=parameter_space,
                                         acquisition=expected_improvement,
                                         batch_size=1)

In [12]:
max_iterations = 30
bayesopt_loop.run_loop(sample, max_iterations)

 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:353: RuntimeWarning:invalid value encountered in divide


fred_compile SEIZ_out.fred ...
No errors found.

FRED Warning (file SEIZ_out.fred) Ignoring duplicate rule:
  if state(SKEPTIC,E) then wait(24*normal(epsilon,epsilon/2))
     is hidden by:
  if state(SKEPTIC,E) then wait()

FRED Warning (file SEIZ_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_0.6064693101466353_0.9158589292284339_2.0_run 2
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/2/WORK/SEIZ_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/2/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/2/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:353: RuntimeWarning:invalid value encountered in divide


fred_compile SEIZ_out.fred ...
No errors found.

FRED Warning (file SEIZ_out.fred) Ignoring duplicate rule:
  if state(SKEPTIC,E) then wait(24*normal(epsilon,epsilon/2))
     is hidden by:
  if state(SKEPTIC,E) then wait()

FRED Warning (file SEIZ_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_0.6832483766671609_0.4324424125634164_2.0_run 3
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/3/WORK/SEIZ_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/3/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/3/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:353: RuntimeWarning:invalid value encountered in divide


...

 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:353: RuntimeWarning:invalid value encountered in divide


fred_compile SEIZ_out.fred ...
No errors found.

FRED Warning (file SEIZ_out.fred) Ignoring duplicate rule:
  if state(SKEPTIC,E) then wait(24*normal(epsilon,epsilon/2))
     is hidden by:
  if state(SKEPTIC,E) then wait()

FRED Warning (file SEIZ_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_0.6688325879969773_0.5021654534999829_2.0_run 30
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/30/WORK/SEIZ_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/30/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/30/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:353: RuntimeWarning:invalid value encountered in divide


fred_compile SEIZ_out.fred ...
No errors found.

FRED Warning (file SEIZ_out.fred) Ignoring duplicate rule:
  if state(SKEPTIC,E) then wait(24*normal(epsilon,epsilon/2))
     is hidden by:
  if state(SKEPTIC,E) then wait()

FRED Warning (file SEIZ_out.fred) No wait rule found for state SKEPTIC.InBetween


FRED Warning (file SEIZ_out.fred) No wait rule found for state ADOPT.InBetween


SEIZ_0.10940609768543186_0.34285769306198643_2.0_run 31
(export OMP_NUM_THREADS=16 ; FRED -p /home/nyoma/FRED/RESULTS/JOB/31/WORK/SEIZ_out.fred -r 1 -d /home/nyoma/FRED/RESULTS/JOB/31/OUT 2>&1 > /home/nyoma/FRED/RESULTS/JOB/31/OUT/RUN1/LOG)


 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/nyoma/.local/lib/python3.9/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/nyoma/.local/lib/python3.9/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


In [13]:
results = bayesopt_loop.get_results()

In [14]:
print(results.minimum_location, results.minimum_value)
print(results.best_found_value_per_iteration)

[0.32229054 0.94554579 2.        ] 9242.0
[13820. 13820. 13820. 10469. 10469. 10469. 10469. 10469. 10469. 10469.
 10469. 10469. 10469. 10469. 10469. 10469. 10469. 10469. 10469.  9242.
  9242.  9242.  9242.  9242.  9242.  9242.  9242.  9242.  9242.  9242.
  9242.  9242.  9242.  9242.  9242.]
